In [1]:
import os
import h5py
import numpy as np
import matplotlib.pyplot as plt
from dataset import *
from torch.utils.data import DataLoader

import torch 
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torchvision import transforms
from torchvision.utils import save_image
from torch.autograd import Variable
import time
from model import *

In [2]:
lr=2e-5
batch_size=1
num_epochs=300
lamda=0.001
sample_dir="./samples"

train_filename='DF_train_data/DF_train.hdf5'
test_filename='DF_test_data/DF_test.hdf5'
train_data=Dataset(train_filename,transform=ToTensor())
test_data=Dataset(test_filename,transform=ToTensor())
train_loader=DataLoader(train_data,batch_size=batch_size,shuffle=False)
test_loader=DataLoader(test_data,batch_size=batch_size,shuffle=False)


In [3]:

Gen1=Generator1().cuda()
Gen2=Generator2().cuda()
Dis=Discriminator().cuda()
g1_optimizer = torch.optim.Adam(Gen1.parameters(), lr = lr, betas=(0.5, 0.999))
g2_optimizer = torch.optim.Adam(Gen2.parameters(), lr = lr, betas=(0.5, 0.999))
d_optimizer   = torch.optim.Adam(Dis.parameters(), lr = lr, betas=(0.5, 0.999))


In [4]:
#loss
def ceriterion_g1(G1_img,tar_img,tar_mask):
    return torch.mean(torch.abs(G1_img-tar_img))+ torch.mean(torch.abs(G1_img-tar_img)*(tar_mask))
def ceriterion_g2(G2_img,tar_img,tar_mask):
    return torch.mean(torch.abs(G2_img-tar_img))+ torch.mean(torch.abs(G2_img-tar_img)*(tar_mask))
ceriterion_d=nn.BCELoss()
ceriterion_g2_bce=nn.BCELoss()


In [ ]:
def denorm(x):
    out = (x+1)/2
    return out.clamp(0, 1)

# For updating learning rate
def update_lr(optimizer, lr):    
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

In [ ]:
curr_lr=lr
start = time.time()
total_step = len(train_loader)
for epochs in range(num_epochs):
    for i, (cond_img, tar_img, cond_pose, tar_pose, cond_mask, tar_mask)in enumerate(train_loader):
        cond_img=Variable(cond_img).cuda()
        tar_img = Variable(tar_img).cuda()
        cond_pose=Variable(cond_pose).cuda()
        tar_pose=Variable(tar_pose).cuda()
        cond_mask=Variable(cond_mask).cuda()
        tar_mask=Variable(tar_mask).cuda()
        G1_img=Gen1.forward(cond_img,tar_pose)
        G2_img=Gen2.forward(cond_img,G1_img)

        g1_loss=ceriterion_g1(G1_img,tar_img,tar_mask)
        #g1 training
        g1_optimizer.zero_grad()
        g1_loss.backward(retain_graph=True)
        g1_optimizer.step()


        #discriminator training
        real_labels = torch.ones(batch_size, 1).cuda()
        fake_labels = torch.zeros(batch_size, 1).cuda()


        #true
        d_outputs=Dis.forward(cond_img,tar_img)
#         print("G1 d output : ",d_outputs)
        d_loss_true=ceriterion_d(d_outputs,real_labels)
        #fake
        d_outputs=Dis.forward(cond_img,G1_img)
#         print("g2 d_output:",d_outputs)
        d_loss_fake=ceriterion_d(d_outputs,fake_labels)
        d_loss=d_loss_true+d_loss_fake
        #d training
        d_optimizer.zero_grad()
        d_loss.backward(retain_graph=True)
        d_optimizer.step()

        #g2 training
        d_g2_outputs=Dis.forward(cond_img,G2_img)
#         print("d_g2_outputs",d_g2_outputs)
        g2_loss_true=ceriterion_g2_bce(d_g2_outputs,real_labels)
        g2_loss_l1=ceriterion_g2(G2_img,tar_img,tar_mask)
        g2_loss=g2_loss_true+g2_loss_l1*lamda
        g2_optimizer.zero_grad()
        g2_loss.backward(retain_graph=True)
        g2_optimizer.step()

    
        if (i+1) % 100 == 0:
            print('Epoch [{}/{}], Step [{}/{}], d_loss: {:.4f}, g1_loss: {:.4f},g2_loss: {:.4f}, '
                  .format(epochs, num_epochs, i+1, total_step, d_loss.item(), g1_loss.item(),g2_loss.item()))
    

     # Save real images

#     save_image(denorm(tar_img), os.path.join(sample_dir, 'real_images-{}.png').format(epochs+1))
#     save_image(denorm(G2_img), os.path.join(sample_dir, 'fake_images-{}.png'.format(epochs+1)))
    save_image(denorm(cond_img), os.path.join(sample_dir, 'img1_cond_images-{}.png').format(epochs+1))
    save_image(denorm(tar_img), os.path.join(sample_dir, 'img1_tar_images-{}.png').format(epochs+1))
    save_image(denorm(G1_img), os.path.join(sample_dir, 'img1_g1_images-{}.png'.format(epochs+1)))
    if epochs % 20 == 0:
        curr_lr /= 3
        update_lr(g1_optimizer, curr_lr)
        update_lr(g2_optimizer, curr_lr)
        update_lr(d_optimizer, curr_lr)
    # Save the model checkpoints
        torch.save(g1_optimizer.state_dict(),'G1_{}.ckpt'.format(epochs+1))
        torch.save(g2_optimizer.state_dict(),'G2_{}.ckpt'.format(epochs+1))
        torch.save(d_optimizer.state_dict(),'D_{}.ckpt'.format(epochs+1))
finished = time.time()
hours = finished-start
print("training finished! %d minutes" % hours)


C:\Users\nhs\Anaconda3\lib\site-packages\torch\nn\modules\upsampling.py:129: UserWarning: nn.Upsample is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.{} is deprecated. Use nn.functional.interpolate instead.".format(self.name))


Epoch [0/300], Step [100/10000], d_loss: 0.0200, g1_loss: 0.8942,g2_loss: 4.4643, 
Epoch [0/300], Step [200/10000], d_loss: 0.0021, g1_loss: 0.7766,g2_loss: 0.0445, 
Epoch [0/300], Step [300/10000], d_loss: 0.0012, g1_loss: 0.8445,g2_loss: 0.0103, 
Epoch [0/300], Step [400/10000], d_loss: 0.0017, g1_loss: 0.8490,g2_loss: 0.0099, 
Epoch [0/300], Step [500/10000], d_loss: 0.0005, g1_loss: 0.8278,g2_loss: 0.0030, 
Epoch [0/300], Step [600/10000], d_loss: 0.0011, g1_loss: 0.8549,g2_loss: 0.0030, 
Epoch [0/300], Step [700/10000], d_loss: 0.0002, g1_loss: 0.7940,g2_loss: 0.0019, 
Epoch [0/300], Step [800/10000], d_loss: 0.0009, g1_loss: 0.7991,g2_loss: 0.0014, 
Epoch [0/300], Step [900/10000], d_loss: 0.0006, g1_loss: 0.3628,g2_loss: 0.0095, 
Epoch [0/300], Step [1000/10000], d_loss: 0.0023, g1_loss: 0.2346,g2_loss: 0.0035, 
Epoch [0/300], Step [1100/10000], d_loss: 0.0014, g1_loss: 0.2364,g2_loss: 0.0082, 
Epoch [0/300], Step [1200/10000], d_loss: 0.0004, g1_loss: 0.3757,g2_loss: 0.0040, 
E

Epoch [0/300], Step [9800/10000], d_loss: 0.0000, g1_loss: 0.1950,g2_loss: 0.0002, 
Epoch [0/300], Step [9900/10000], d_loss: 0.0000, g1_loss: 0.3121,g2_loss: 0.0004, 
Epoch [0/300], Step [10000/10000], d_loss: 0.0000, g1_loss: 0.2713,g2_loss: 0.0003, 
Epoch [1/300], Step [100/10000], d_loss: 0.0000, g1_loss: 0.2685,g2_loss: 0.0003, 
Epoch [1/300], Step [200/10000], d_loss: 0.0000, g1_loss: 0.1955,g2_loss: 0.0002, 
Epoch [1/300], Step [300/10000], d_loss: 0.0000, g1_loss: 0.2510,g2_loss: 0.0003, 
Epoch [1/300], Step [400/10000], d_loss: 0.0000, g1_loss: 0.2397,g2_loss: 0.0003, 
Epoch [1/300], Step [500/10000], d_loss: 0.0000, g1_loss: 0.2550,g2_loss: 0.0003, 
Epoch [1/300], Step [600/10000], d_loss: 0.0000, g1_loss: 0.3140,g2_loss: 0.0004, 
Epoch [1/300], Step [700/10000], d_loss: 0.0000, g1_loss: 0.3793,g2_loss: 0.0005, 
Epoch [1/300], Step [800/10000], d_loss: 0.0000, g1_loss: 0.2907,g2_loss: 0.0003, 
Epoch [1/300], Step [900/10000], d_loss: 0.0000, g1_loss: 0.2525,g2_loss: 0.0003, 


Epoch [1/300], Step [9500/10000], d_loss: 0.0000, g1_loss: 0.1917,g2_loss: 0.0002, 
Epoch [1/300], Step [9600/10000], d_loss: 0.0000, g1_loss: 0.1623,g2_loss: 0.0002, 
Epoch [1/300], Step [9700/10000], d_loss: 0.0000, g1_loss: 0.2705,g2_loss: 0.0003, 
Epoch [1/300], Step [9800/10000], d_loss: 0.0000, g1_loss: 0.1866,g2_loss: 0.0002, 
Epoch [1/300], Step [9900/10000], d_loss: 0.0000, g1_loss: 0.2968,g2_loss: 0.0003, 
Epoch [1/300], Step [10000/10000], d_loss: 0.0000, g1_loss: 0.2656,g2_loss: 0.0003, 
Epoch [2/300], Step [100/10000], d_loss: 0.0000, g1_loss: 0.2725,g2_loss: 0.0003, 
Epoch [2/300], Step [200/10000], d_loss: 0.0000, g1_loss: 0.1732,g2_loss: 0.0002, 
Epoch [2/300], Step [300/10000], d_loss: 0.0000, g1_loss: 0.2536,g2_loss: 0.0003, 
Epoch [2/300], Step [400/10000], d_loss: 0.0000, g1_loss: 0.2431,g2_loss: 0.0003, 
Epoch [2/300], Step [500/10000], d_loss: 0.0000, g1_loss: 0.2492,g2_loss: 0.0003, 
Epoch [2/300], Step [600/10000], d_loss: 0.0000, g1_loss: 0.3167,g2_loss: 0.0003

Epoch [2/300], Step [9200/10000], d_loss: 0.0000, g1_loss: 0.2897,g2_loss: 0.0003, 
Epoch [2/300], Step [9300/10000], d_loss: 0.0000, g1_loss: 0.2283,g2_loss: 0.0002, 
Epoch [2/300], Step [9400/10000], d_loss: 0.0000, g1_loss: 0.3016,g2_loss: 0.0003, 
Epoch [2/300], Step [9500/10000], d_loss: 0.0000, g1_loss: 0.1910,g2_loss: 0.0002, 
Epoch [2/300], Step [9600/10000], d_loss: 0.0000, g1_loss: 0.1605,g2_loss: 0.0002, 
Epoch [2/300], Step [9700/10000], d_loss: 0.0000, g1_loss: 0.2681,g2_loss: 0.0003, 
Epoch [2/300], Step [9800/10000], d_loss: 0.0000, g1_loss: 0.1835,g2_loss: 0.0002, 
Epoch [2/300], Step [9900/10000], d_loss: 0.0000, g1_loss: 0.2912,g2_loss: 0.0003, 
Epoch [2/300], Step [10000/10000], d_loss: 0.0000, g1_loss: 0.2717,g2_loss: 0.0003, 
Epoch [3/300], Step [100/10000], d_loss: 0.0000, g1_loss: 0.2724,g2_loss: 0.0003, 
Epoch [3/300], Step [200/10000], d_loss: 0.0000, g1_loss: 0.1713,g2_loss: 0.0002, 
Epoch [3/300], Step [300/10000], d_loss: 0.0000, g1_loss: 0.2463,g2_loss: 0.0

...

Epoch [136/300], Step [500/10000], d_loss: 0.0000, g1_loss: 0.2111,g2_loss: 0.0002, 
Epoch [136/300], Step [600/10000], d_loss: 0.0000, g1_loss: 0.1557,g2_loss: 0.0002, 
Epoch [136/300], Step [700/10000], d_loss: 0.0000, g1_loss: 0.2401,g2_loss: 0.0002, 
Epoch [136/300], Step [800/10000], d_loss: 0.0000, g1_loss: 0.1775,g2_loss: 0.0002, 
Epoch [136/300], Step [900/10000], d_loss: 0.0000, g1_loss: 0.1727,g2_loss: 0.0002, 
Epoch [136/300], Step [1000/10000], d_loss: 0.0000, g1_loss: 0.1201,g2_loss: 0.0001, 
Epoch [136/300], Step [1100/10000], d_loss: 0.0000, g1_loss: 0.0951,g2_loss: 0.0001, 
Epoch [136/300], Step [1200/10000], d_loss: 0.0000, g1_loss: 0.2081,g2_loss: 0.0002, 
Epoch [136/300], Step [1300/10000], d_loss: 0.0000, g1_loss: 0.1149,g2_loss: 0.0001, 
Epoch [136/300], Step [1400/10000], d_loss: 0.0000, g1_loss: 0.1946,g2_loss: 0.0002, 
Epoch [136/300], Step [1500/10000], d_loss: 0.0000, g1_loss: 0.1718,g2_loss: 0.0002, 
Epoch [136/300], Step [1600/10000], d_loss: 0.0000, g1_loss

Epoch [136/300], Step [10000/10000], d_loss: 0.0000, g1_loss: 0.1546,g2_loss: 0.0002, 
Epoch [137/300], Step [100/10000], d_loss: 0.0000, g1_loss: 0.1919,g2_loss: 0.0002, 
Epoch [137/300], Step [200/10000], d_loss: 0.0000, g1_loss: 0.1018,g2_loss: 0.0001, 
Epoch [137/300], Step [300/10000], d_loss: 0.0000, g1_loss: 0.1440,g2_loss: 0.0001, 
Epoch [137/300], Step [400/10000], d_loss: 0.0000, g1_loss: 0.1205,g2_loss: 0.0001, 
Epoch [137/300], Step [500/10000], d_loss: 0.0000, g1_loss: 0.2111,g2_loss: 0.0002, 
Epoch [137/300], Step [600/10000], d_loss: 0.0000, g1_loss: 0.1557,g2_loss: 0.0002, 
Epoch [137/300], Step [700/10000], d_loss: 0.0000, g1_loss: 0.2401,g2_loss: 0.0002, 
Epoch [137/300], Step [800/10000], d_loss: 0.0000, g1_loss: 0.1775,g2_loss: 0.0002, 
Epoch [137/300], Step [900/10000], d_loss: 0.0000, g1_loss: 0.1727,g2_loss: 0.0002, 
Epoch [137/300], Step [1000/10000], d_loss: 0.0000, g1_loss: 0.1201,g2_loss: 0.0001, 
Epoch [137/300], Step [1100/10000], d_loss: 0.0000, g1_loss: 0

Epoch [137/300], Step [9500/10000], d_loss: 0.0000, g1_loss: 0.1444,g2_loss: 0.0001, 
Epoch [137/300], Step [9600/10000], d_loss: 0.0000, g1_loss: 0.1224,g2_loss: 0.0001, 
Epoch [137/300], Step [9700/10000], d_loss: 0.0000, g1_loss: 0.1575,g2_loss: 0.0002, 
Epoch [137/300], Step [9800/10000], d_loss: 0.0000, g1_loss: 0.1306,g2_loss: 0.0001, 
Epoch [137/300], Step [9900/10000], d_loss: 0.0000, g1_loss: 0.2120,g2_loss: 0.0002, 
Epoch [137/300], Step [10000/10000], d_loss: 0.0000, g1_loss: 0.1546,g2_loss: 0.0002, 
Epoch [138/300], Step [100/10000], d_loss: 0.0000, g1_loss: 0.1919,g2_loss: 0.0002, 
Epoch [138/300], Step [200/10000], d_loss: 0.0000, g1_loss: 0.1018,g2_loss: 0.0001, 
Epoch [138/300], Step [300/10000], d_loss: 0.0000, g1_loss: 0.1440,g2_loss: 0.0001, 
Epoch [138/300], Step [400/10000], d_loss: 0.0000, g1_loss: 0.1205,g2_loss: 0.0001, 
Epoch [138/300], Step [500/10000], d_loss: 0.0000, g1_loss: 0.2111,g2_loss: 0.0002, 
Epoch [138/300], Step [600/10000], d_loss: 0.0000, g1_loss

Epoch [138/300], Step [9000/10000], d_loss: 0.0000, g1_loss: 0.1610,g2_loss: 0.0002, 
Epoch [138/300], Step [9100/10000], d_loss: 0.0000, g1_loss: 0.1682,g2_loss: 0.0002, 
Epoch [138/300], Step [9200/10000], d_loss: 0.0000, g1_loss: 0.1945,g2_loss: 0.0002, 
Epoch [138/300], Step [9300/10000], d_loss: 0.0000, g1_loss: 0.1536,g2_loss: 0.0002, 
Epoch [138/300], Step [9400/10000], d_loss: 0.0000, g1_loss: 0.1930,g2_loss: 0.0002, 
Epoch [138/300], Step [9500/10000], d_loss: 0.0000, g1_loss: 0.1444,g2_loss: 0.0001, 
Epoch [138/300], Step [9600/10000], d_loss: 0.0000, g1_loss: 0.1224,g2_loss: 0.0001, 
Epoch [138/300], Step [9700/10000], d_loss: 0.0000, g1_loss: 0.1575,g2_loss: 0.0002, 
Epoch [138/300], Step [9800/10000], d_loss: 0.0000, g1_loss: 0.1306,g2_loss: 0.0001, 
Epoch [138/300], Step [9900/10000], d_loss: 0.0000, g1_loss: 0.2120,g2_loss: 0.0002, 
Epoch [138/300], Step [10000/10000], d_loss: 0.0000, g1_loss: 0.1546,g2_loss: 0.0002, 
Epoch [139/300], Step [100/10000], d_loss: 0.0000, g1

Epoch [139/300], Step [8500/10000], d_loss: 0.0000, g1_loss: 0.2798,g2_loss: 0.0003, 
Epoch [139/300], Step [8600/10000], d_loss: 0.0000, g1_loss: 0.1762,g2_loss: 0.0002, 
Epoch [139/300], Step [8700/10000], d_loss: 0.0000, g1_loss: 0.2101,g2_loss: 0.0002, 
Epoch [139/300], Step [8800/10000], d_loss: 0.0000, g1_loss: 0.1651,g2_loss: 0.0002, 
Epoch [139/300], Step [8900/10000], d_loss: 0.0000, g1_loss: 0.1969,g2_loss: 0.0002, 
Epoch [139/300], Step [9000/10000], d_loss: 0.0000, g1_loss: 0.1610,g2_loss: 0.0002, 
Epoch [139/300], Step [9100/10000], d_loss: 0.0000, g1_loss: 0.1682,g2_loss: 0.0002, 
Epoch [139/300], Step [9200/10000], d_loss: 0.0000, g1_loss: 0.1944,g2_loss: 0.0002, 
Epoch [139/300], Step [9300/10000], d_loss: 0.0000, g1_loss: 0.1536,g2_loss: 0.0002, 
Epoch [139/300], Step [9400/10000], d_loss: 0.0000, g1_loss: 0.1930,g2_loss: 0.0002, 
Epoch [139/300], Step [9500/10000], d_loss: 0.0000, g1_loss: 0.1443,g2_loss: 0.0001, 
Epoch [139/300], Step [9600/10000], d_loss: 0.0000, g1